In [1]:
#파이프 라인 
# 데이터 전처리 및 모델링 코드를 체계적으로 유지하는 간단한 방법
# 파이프라인은 전처리 및 모델링 단계를 번들로 묶어 전체 번들을 단일 단계인 것처럼 사용할 수 있음

#깔끔한 코드 / 버그 감소/ 쉬워진 생산화 / 모델 검증을 위한 추가 옵션



In [3]:
import pandas as pd 
from sklearn.model_selection import train_test_split

data = pd.read_csv('../../data/melb_data.csv')

# 독립변수 / 종속변수 분리 
y = data.Price  #종속변수
x = data.drop(['Price'], axis=1)

#훈련용 / 검증용 데이터 분리
x_train, x_test, y_train, y_test =\
train_test_split(x, y, train_size=0.8, test_size=0.2, random_state=0)

In [4]:
#훈련데이터의 열의 고윳값이 10보다 작고 데이터 타입이 object인 열 >> low_cardinality_cols
cardinality_cols = [cname for cname in x_train.columns if x_train[cname].nunique() < 10
                        and x_train[cname].dtype == 'object']

#숫자형 데이터
numerical_cols = [cname for cname in x_train.columns if x_train[cname].dtype in ['int64', 'float64']]

# 선택한 열만 유지 
my_cols = cardinality_cols + numerical_cols
x_train = x_train[my_cols].copy()
x_test = x_test[my_cols].copy()

In [8]:
# 1. 전처리 단계 준비 
# 숫자 데이터의 결측값을 대치
# 누락된 값을 대치하고 범주형 데이터에 원-핫 인코딩을 적용

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

#결측값 대치 모델
numerical_transformer = SimpleImputer(strategy='constant')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

#숫자 및 범주형 데이터에 대한 번들 전처리
preprocessor = ColumnTransformer(
    transformers = [
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, cardinality_cols)
    ]
)
preprocessor


ColumnTransformer(transformers=[('num', SimpleImputer(strategy='constant'),
                                 ['Rooms', 'Distance', 'Postcode', 'Bedroom2',
                                  'Bathroom', 'Car', 'Landsize', 'BuildingArea',
                                  'YearBuilt', 'Lattitude', 'Longtitude',
                                  'Propertycount']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Type', 'Method', 'Regionname'])])

In [9]:
# 2. 모델정의 (랜덤포레스트모델을 정의 )
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

In [10]:
# 3. 파이프라인 생성 및 평가 
#Pipeline 클래스를 사용하여 전처리 및 모델링 단계를 묶는 파이프라인을 정의

from sklearn.metrics import mean_absolute_error

my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model',model)])

my_pipeline.fit(x_train, y_train)

preds = my_pipeline.predict(x_test)

score = mean_absolute_error(y_test, preds)
print('MAE:', score)

MAE: 160679.18917034855
